In [2]:
import urllib
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import json

In [4]:
import openapi

In [5]:
import requests

service_key = "l53cN3mbhsudCVoI7zttwvpHkgCTM8on0LpEaT2FZT1/6+weHhppLkhYpjiYtdPL3/h7rqYRcFNBJQCklXUZUQ=="

url = 'http://apis.data.go.kr/B551172/Gastric01/patientNumber'

params ={'serviceKey' : service_key, 
        'type' : 'json', 
        'numOfRows' : 100, 
        'centerNm' : '국립암센터', 
        'fromYear' : '2010', 
        'toYear' : '2019' }

all_data = []

# 1️⃣ 먼저 첫 번째 요청을 보내서 totalCount 확인
response = requests.get(url, params=params)

if response.status_code == 200:
    resp_dict = response.json()

    # 🔹 totalCount가 없을 경우 기본값 0으로 설정 (오류 방지)
    total_count = int(resp_dict.get('totalCount', 0) or 0)  
    items_per_page = params['numOfRows']  
    total_pages = (total_count // items_per_page) + (1 if total_count % items_per_page else 0)

    print(f"📌 총 데이터 개수: {total_count}, 가져올 페이지 수: {total_pages}")

    # ✅ 데이터가 없을 경우 메시지 출력 후 종료
    if total_count == 0:
        print("⚠️ 가져올 데이터가 없습니다.")
    else:
        # 2️⃣ total_pages 만큼 반복하여 모든 데이터를 가져오기
        for i in range(1, total_pages + 1):
            page_params = params.copy()  # 🔹 원본 params 보호
            page_params['pageNo'] = i  

            response = requests.get(url, params=page_params)

            if response.status_code == 200:
                resp_dict = response.json()
                items = resp_dict.get('items', [])  # 🔹 데이터가 없을 경우 빈 리스트 반환
                all_data.extend(items)
                print(f"📄 페이지 {i}/{total_pages} 처리 완료! (누적 {len(all_data)}개)")
            else:
                print(f"❌ 데이터 요청 실패 (페이지 {i})")

        print(f"✅ 총 {len(all_data)}개의 데이터 수집 완료!")

else:
    print(f"❌ API 요청 실패 - HTTP 상태 코드: {response.status_code}")

📌 총 데이터 개수: 1116, 가져올 페이지 수: 12
📄 페이지 1/12 처리 완료! (누적 100개)
📄 페이지 2/12 처리 완료! (누적 200개)
📄 페이지 3/12 처리 완료! (누적 300개)
📄 페이지 4/12 처리 완료! (누적 400개)
📄 페이지 5/12 처리 완료! (누적 500개)
📄 페이지 6/12 처리 완료! (누적 600개)
📄 페이지 7/12 처리 완료! (누적 700개)
📄 페이지 8/12 처리 완료! (누적 800개)
📄 페이지 9/12 처리 완료! (누적 900개)
📄 페이지 10/12 처리 완료! (누적 1000개)
📄 페이지 11/12 처리 완료! (누적 1100개)
📄 페이지 12/12 처리 완료! (누적 1116개)
✅ 총 1116개의 데이터 수집 완료!


In [6]:
df_stomach_cancer = pd.DataFrame(all_data)
df_stomach_cancer

statsMetaNo centerNm critYr ptAge ptSexCd statsTrgtNm ncsNmvl  \
0          17010    국립암센터   2016    64       M                   0   
1          17010    국립암센터   2019    75       M                   0   
2          17010    국립암센터   2012    77       M                   0   
3          17010    국립암센터   2011    74       F                   0   
4          17010    국립암센터   2010    56       M                   0   
...          ...      ...    ...   ...     ...         ...     ...   
1111       17010    국립암센터   2013    37       M                   0   
1112       17010    국립암센터   2019    82       F                   0   
1113       17010    국립암센터   2016    84       F                   0   
1114       17010    국립암센터   2017    37       M                   0   
1115       17010    국립암센터   2016    88       M                   0   

     wholNcsDnmvl ptCntNmvl wholPtCntDnmvl  
0               0        16              0  
1               0        12              0  
2               0         8              0  
3               0        10              0  
4               0        16              0  
...           ...       ...            ...  
1111            0         2              0  
1112            0         3              0  
1113            0         2              0  
1114            0         1              0  
1115            0         1              0  

[1116 rows x 10 columns]

In [ ]:
df_scancer = df_stomach_cancer.iloc[:, [1,2,3,4,-2]]

In [8]:
df_scancer.rename(columns={'ptAge': '나이'}, inplace=True)

C:\Users\human\AppData\Local\Temp\ipykernel_19680\2667697859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scancer.rename(columns={'ptAge': '나이'}, inplace=True)


In [9]:
df_scancer.rename(columns={'ptSexCd': "성별"}, inplace=True)

C:\Users\human\AppData\Local\Temp\ipykernel_19680\3103237396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scancer.rename(columns={'ptSexCd': "성별"}, inplace=True)


In [10]:
df_scancer.rename(columns={'ptCntNmvl': "환자수"}, inplace=True)

C:\Users\human\AppData\Local\Temp\ipykernel_19680\1536472180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scancer.rename(columns={'ptCntNmvl': "환자수"}, inplace=True)


In [11]:
df_scancer

centerNm critYr  나이 성별 환자수
0       국립암센터   2016  64  M  16
1       국립암센터   2019  75  M  12
2       국립암센터   2012  77  M   8
3       국립암센터   2011  74  F  10
4       국립암센터   2010  56  M  16
...       ...    ...  .. ..  ..
1111    국립암센터   2013  37  M   2
1112    국립암센터   2019  82  F   3
1113    국립암센터   2016  84  F   2
1114    국립암센터   2017  37  M   1
1115    국립암센터   2016  88  M   1

[1116 rows x 5 columns]

In [12]:
df_scancer['성별'].value_counts()

성별
F    559
M    557
Name: count, dtype: int64

In [13]:
total_patients = df_scancer['환자수'].astype(int).sum()
total_patients

np.int64(6641)